In [18]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack
import urllib3
import IPython, time
import os

file_name = "a"
path = file_name +".bmp"

print("get matrix image..........")
def get_matrix_image(url):
    im = Image.open(url)
    np_im = np.array(im)
    return np_im
        
matrix = get_matrix_image(path)

hist = {}
high = matrix.shape[0]
weight = matrix.shape[1]
kok = matrix.shape[2]

print("get hist...........")
for i in range(0, high):
    for j in range(0, weight):
        for k in range(0, kok):
            hist[matrix[i][j][k]] =  hist.get(matrix[i][j][k],0) + 1

print("sort frequency...............")
def sortFreq (vector) :
    value = vector.keys()
    tuples = []
    for i in value :
        tuples.append((vector[i],i))
    tuples.sort()
    return tuples
sorted_hist = sortFreq(hist)

def getKey(tuple):
    return tuple[0]
def getValue(tuple):
    return tuple[1]
print("build tree............")
def buildTree(vector):
    while len(vector) > 1:
        lowestTwo = tuple(vector[0:2])
        theRest = vector[2:]
        sumPro = lowestTwo[0][0] + lowestTwo[1][0]
        vector = theRest + [(sumPro, lowestTwo)]
        sorted(vector, key = getKey)
    return vector[0]

tree = buildTree(sorted_hist)
a = 3
a = np.dtype('uint8').type(a)
def Tree(tree):
    p = tree[1]
    if type(p) == type(a): 
        return p
    else:
        return (trimTree(p[0]), trimTree(p[1]))
trim = Tree(tree)
code ={}

def assignCodes(n, pat = ''):
    if type(n) == type(a):
        code[n] = pat
    else:
        assignCodes(n[0], pat+"0")
        assignCodes(n[1], pat+"1")
        
assignCodes(trim)

def pad_encoded_text(encoded_text):
    extra_padding = 8 - len(encoded_text) % 8
    for i in range(extra_padding):
        encoded_text += "0"

    padded_info = "{0:08b}".format(extra_padding)
    encoded_text = padded_info + encoded_text
    return encoded_text
def get_byte_array(padded_encoded_text):
    if(len(padded_encoded_text) % 8 != 0):
        print("Encoded text not padded properly")
        exit(0)

    b = bytearray()
    for i in range(0, len(padded_encoded_text), 8):
        byte = padded_encoded_text[i:i+8]
        b.append(int(byte, 2))
    return b

print("encoding.....................")

def encode(vector):
    file = open("text_code.txt","w+")
    start_time = time.time()
    for i in range(0, high):
        for j in range(0, weight):
            for k in range(0, kok):
                file.write(code[matrix[i][j][k]])
    file.close()
def read():
    file = open("text_code.txt", "r")
    t = pad_encoded_text(file.read())
    file.close()
    b = get_byte_array(t)
    file = open("out.bin","wb")
    file.write(bytes(b))
    file.close()
    
encode(matrix)
read()
end_time = time.time()
print ('thời gian chạy encode: %f ms' % ((end_time - start_time) * 1000))
def read_file(filename):
    file = open(filename, 'rb')
    bit_string = ""
    byte = file.read()
    for i in byte:
        bits = bin(i)[2:].rjust(8, '0')
        bit_string += bits
    # loại bỏ các ký tự fix size ở cuối của string 
    fixed_size = bit_string[:8]
    fixed_size = int(fixed_size, 2)
    encoded_text = bit_string[8:] 
    encoded_text = encoded_text[:-1*fixed_size]
    # trả về string nhị phân
    return encoded_text
    pass

bit_string = read_file("out.bin")
print("decoding.............")
def decode(tree, str):
    output = np.zeros((high,weight,3))
    output = np.uint8(output)
    k =0
    j = 0
    n = 0
    p = tree
    for i in str:
        if j == weight:
            j = 0
            k+=1
        if n == 3:
            j+=1
            n = 0
        if i == '0': p = p[0]
        else: p = p[1]
        if type(p) == type(a):
            p = np.dtype('uint8').type(p)
            output[k][j][n] = p
            n+=1
            p = tree
    return output

ot = decode(trim, bit_string)
ot = np.array(ot)
new_im = Image.fromarray(ot)
new_im.save(file_name+"_decode.png")
new_im.show()
os.remove("text_code.txt")
print("finish")

get matrix image..........
get hist...........
sort frequency...............
build tree............
encoding.....................
thời gian chạy encode: 455216.823339 ms
decoding.............
finish
